## Speed up.

In [ ]:
from shared.preface import *
from shared.shared_functions import *

# delete_temp_data('L025N752/DMONLY/SigmaConstant00/all_sky_TEST/temp_data_TEST/nu_*.npy')

directory = f'L025N752/DMONLY/SigmaConstant00/all_sky_TEST'

shell_multipliers = np.load(f'{directory}/shell_multipliers.npy')
FCT_shell_multipliers = np.copy(shell_multipliers)

def cell_gravity_short_range(
    cell_coords_in, cell_gen, init_GRID_S,
    DM_pos, DM_lim, DM_sim_mass, smooth_l,
    out_dir, b_id, max_b_len
):

    # Cell lengths to limit DM particles. Limit for the largest cell is 
    # GRID_S/2, not just GRID_S, therefore the cell_gen+1
    cell_len = init_GRID_S / (2 ** (cell_gen + 1))

    # Center all DM positions w.r.t. cell center.
    # DM_pos already in shape = (1, DM_particles, 3)
    DM_pos_cent = DM_pos - cell_coords_in[:, np.newaxis, :]

    # Select DM particles inside each cell based on cube length generation
    DM_in_cell_IDs = (
        (np.abs(DM_pos_cent[..., 0]) < cell_len[:, np.newaxis]) &
        (np.abs(DM_pos_cent[..., 1]) < cell_len[:, np.newaxis]) &
        (np.abs(DM_pos_cent[..., 2]) < cell_len[:, np.newaxis])
    )

    # Set DM outside cell to nan values.
    DM_pos_cent[~DM_in_cell_IDs] = np.nan

    # Save the DM IDs, such that we know which particles are in which cell.
    # This will be used in the long-range gravity calculations.
    DM_in_cell_IDs_compact = np.argwhere(DM_in_cell_IDs)
    DM_in_cell_IDs_compact[:, 0] += max_b_len * b_id
    np.save(f'{out_dir}/batch{b_id}_DM_in_cell_IDs.npy', DM_in_cell_IDs_compact)
    del DM_in_cell_IDs_compact

    # Sort all nan values to the bottom of axis 1, i.e. the DM-in-cell-X axis 
    # and truncate array based on DM_lim parameter. This simple way works since 
    # each cell cannot have more than DM_lim (times the last shell multiplier).
    ind_2D = DM_pos_cent[:,:,0].argsort(axis=1)
    ind_3D = np.repeat(np.expand_dims(ind_2D, axis=2), 3, axis=2)
    DM_sort = np.take_along_axis(DM_pos_cent, ind_3D, axis=1)
    DM_in = DM_sort[:,:DM_lim*FCT_shell_multipliers[-1],:]

    # Calculate distances of DM to respective cell center.
    DM_dis = np.sqrt(np.sum(DM_in**2, axis=2))

    # Offset DM positions by smoothening length of Camila's simulations.
    eps = smooth_l / 2.

    # Quotient in sum (see formula). Can contain nan values, thus the np.nansum 
    # for the derivative, s.t. these values don't contribute. We center DM on 
    # c.o.m. of cell C, so we only need DM_in in numerator.
    quot = (-DM_in) / ((DM_dis[..., np.newaxis]**2 + eps**2)**(3/2))
    derivative = -G * DM_sim_mass * np.nansum(quot, axis=1)
    np.save(f'{out_dir}/batch{b_id}_short_range.npy', derivative)

cell_coords_in = 
cell_gen
init_GRID_S
DM_pos
DM_lim = 10_000
DM_sim_mass
smooth_l
out_dir
b_id
max_b_len

cell_gravity_short_range(
    cell_coords_in, cell_gen, init_GRID_S,
    DM_pos, DM_lim, DM_sim_mass, smooth_l,
    out_dir, b_id, max_b_len
)

## Snaprange

In [ ]:
from shared.preface import *
from shared.shared_functions import *

nums_snaps = np.load('L025N752/DMONLY/SigmaConstant00/all_sky_analytical_tests/nums_snaps.npy')

def make_snap_range(snap_start, snap_stop):
    snap_range = [f'{s:04d}' for s in range(snap_start, snap_stop+1)]
    return np.array(snap_range)

snap_range = make_snap_range(snap_start=34,snap_stop=36)

snap_IDs = np.array([np.where(s == nums_snaps) for s in snap_range]).flatten()
print(snap_IDs)

## Generic 3D plot.

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

cond = ()
x_DM, y_DM, z_DM = DM_pos_orig[:,0], DM_pos_orig[:,1], DM_pos_orig[:,2]
cut = 1
x, y, z = x_DM[1::cut], y_DM[1::cut], z_DM[1::cut]
ax.scatter(x, y, z, alpha=1, c='blueviolet', s=0.0001, label='DM particles')
# ax.scatter(0.465, 0.025, 8.48, c='green', s=100, label='Earth')
axlim = 20
# ax.set_xlabel('x-axis?')
ax.set_xlim(-axlim, axlim)
ax.set_ylim(-axlim, axlim)
ax.set_zlim(-axlim, axlim)
ax.view_init(elev=0, azim=0)
plt.legend(loc='upper right')
plt.show()

## Quiver plot for directionality.

In [ ]:
from shared.preface import *
from shared.shared_functions import *


reso = 'low'
init_vels = np.load(
    f'L025N752/DMONLY/SigmaConstant00/{reso}_res_all_sky/initial_velocities.npy'
)
ic(init_vels.shape)

def vel_arrows(
    pos_coords, vel_coords
):

    # Coordinates in position space.
    pX = pos_coords[:,0]
    pY = pos_coords[:,1]
    pZ = pos_coords[:,2]

    # Coordinates in velocity space.

    vX = vel_coords[:,0]
    vY = vel_coords[:,1]
    vZ = vel_coords[:,2]

    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    start, stop = 0, -1 
    # start, stop = 100, -1 
    ax.quiver(
        pX[start:stop], pY[start:stop], pZ[start:stop], 
        vX[start:stop], vY[start:stop], vZ[start:stop], 
        length=0.05, normalize=True
    )
    ax.set_xlabel('x-axis')
    ax.set_ylabel('y-axis')
    ax.set_zlabel('z-axis')
    ax.view_init(elev=90, azim=0)

    plt.show()


vel_coords = init_vels[:,0,:]
vel_coords = vel_coords[vel_coords[...,0] > 0.]
pos_coords = np.zeros((len(vel_coords),3))
vel_arrows(pos_coords, vel_coords)

## Healpix conventions: pixel numbering and axes.

In [ ]:
from shared.preface import *
from shared.shared_functions import *

Nside = 2**2  # Specified nside parameter, power of 2
Npix = 12 * Nside**2  # Number of pixels
pix_sr = (4*np.pi)/Npix  # Pixel size  [sr]

'''
### Testing DM projection. ###
### ---------------------- ###
DM_test = np.array([16, -8, 8])
obs_xyz = np.array([8, 0, 0])
DM_obs_cent = DM_test - obs_xyz
DM_obs_cent[0] *= -1  #! important
DM_proj_XY_plane_dis = np.sqrt(np.sum(DM_obs_cent[:2]**2, axis=-1))
DM_obs_dis = np.sqrt(np.sum(DM_obs_cent**2, axis=-1))

thetas = np.arctan2(DM_obs_cent[2], DM_proj_XY_plane_dis)
phis = np.arctan2(DM_obs_cent[1], DM_obs_cent[0])

hp_glon, hp_glat = np.rad2deg(phis), np.rad2deg(thetas)

# Convert angles to pixel indices using ang2pix.
pixel_indices = hp.ang2pix(Nside, hp_glon, hp_glat, lonlat=True)

# Increment the corresponding pixels of empty healpix map.
healpix_map = np.zeros(Npix)
np.add.at(healpix_map, pixel_indices, 1)

hp.newvisufunc.projview(
    healpix_map,
    coord=['G'],
    title=f'Test',
    unit=r'',
    graticule=True,
    graticule_labels=True,
    xlabel="longitude",
    ylabel="latitude",
    cb_orientation="horizontal",
    projection_type="mollweide",
    flip='astro'
)
# '''


# '''
### Testing initial velocities. ###
### --------------------------- ###

# Galactic coordinates.
phi_angles, theta_angles = np.array(
    hp.pixelfunc.pix2ang(Nside, np.arange(Npix), lonlat=True)
)
# ic(phi_angles)
# ic(theta_angles)

bt = 0
lt = Pi/4
#? make sure to only use positive l
theta_angles = theta_angles[np.abs(theta_angles - np.rad2deg(bt)).argmin()]
phi_angles = phi_angles[np.abs(phi_angles - np.rad2deg(lt)).argmin()]

glat = np.deg2rad(theta_angles)
glon = np.deg2rad(phi_angles)
uX = np.squeeze(np.array([np.cos(glat)*np.cos(glon)]))
uY = np.squeeze(np.array([np.cos(glat)*np.sin(glon)]))
uZ = np.squeeze(np.array([np.sin(glat)]))
u_proj_XY_plane_dis = np.sqrt(uX**2 + uY**2)

ic(f'{uX:.2f}') 
ic(f'{uY:.2f}')
ic(f'{uZ:.2f}')


#? these are the angles for the healpy projection. Are the axes (directions of uX, uY, uZ) consistent with the simulation frame?

thetas = np.arctan2(uZ, u_proj_XY_plane_dis)
phis = np.arctan2(uY, uX)

hp_glon, hp_glat = np.rad2deg(phis), np.rad2deg(thetas)

# Convert angles to pixel indices using ang2pix.
pixel_indices = hp.ang2pix(Nside, hp_glon, hp_glat, lonlat=True)

# Increment the corresponding pixels of empty healpix map.
healpix_map2 = np.zeros(Npix)
np.add.at(healpix_map2, pixel_indices, 1)

hp.newvisufunc.projview(
    healpix_map2,
    coord=['G'],
    title=f'Test',
    unit=r'',
    graticule=True,
    graticule_labels=True,
    xlabel="longitude",
    ylabel="latitude",
    cb_orientation="horizontal",
    projection_type="mollweide",
    flip='astro'
)
# '''

## Rotation matrices for DM projection to healpix map.

In [ ]:
from shared.preface import *
from shared.shared_functions import *

init_dis = 8.5
orig_xyz = np.array([init_dis, 0, 0.])
ic(orig_xyz)
xE = np.cos(np.deg2rad(Pi))*np.sin(np.deg2rad(Pi))*init_dis
yE = np.sin(np.deg2rad(Pi))*np.sin(np.deg2rad(Pi))*init_dis
zE = np.cos(np.deg2rad(Pi))*init_dis
init_xyz = np.array([xE, yE, zE])
# ic(init_xyz)

# Rotations happen in clockwise direction, when viewed from the positive top of 
# the axis around which rotation happends. 
# note: See GoodNotes drawing, for orientation of axes the functions assume.

# Earth was on x-axis originally. Applying matrix with positive angles will 
# result in going to the new earth frame, where now the original position is 
# seen from earths frame.
# This matrix must be used, to get all the DM in this new frame, such that we 
# can project it on the healpix map#!
order = 'zyx'
z_rot = Pi
y_rot = -Pi
x_rot = 0
rotation_mat_for_map = rotation_matrix(z_rot, y_rot, x_rot, order)
# ic(rotation_mat)
# angles = rotation_angles(rotation_mat, order)
# print(angles)
rot_xyz = np.matmul(rotation_mat_for_map, orig_xyz)
ic(rot_xyz)

# Rotation matrix with negative angles must be used, to get earths slightly 
# shifted position (needed for the cell edge issue) in the original frame, 
# where all the simulation happens#!
rotation_mat_for_sim = rotation_matrix(-z_rot, -y_rot, -x_rot, order)


### TEST ###

# Take a DM particle in the original frame with coordinates (x,y,z)_DM and 
# earth position in the same frame with (x,y,z)_E, where Earth is not 
# positioned on the x-axis. The only thing that we have to do, to "rotate" the 
# DM particle into the frame, where Earth is again on the x-axis connecting it 
# to the halo center, is to multiply (x,y,z)_DM by the rotation matrix, which 
# brought earth to be at the location in the original frame in the first place.

# note: We can get the galactic latitude and galactic longitude in the frame, where Earth lies on the x-axis (see drawings) connecting it to the halo center.

# Step 1: Original positions.
DM_test = np.array([-5, 1, 5])  # DM particle in original frame

# Offset Earth position from x-axis (because of the cell edge issue).
rot_mat = rotation_matrix(0, 0, 20, order)

# DM particle in earths frame.
DM_earth = np.matmul(rot_mat, DM_test)
ic('************')
ic(DM_test)
ic(DM_earth)

## Visual of all colorcet maps.

## Old stuff: to be analyzed and documented.

In [ ]:
def nu_in_which_cell(x_i, cell_coords, cell_gens, init_GRID_S):

    # print(x_i.shape)  # (3,)
    # print(cell_coords.shape)  # (cells, 1, 3)
    # print(cell_gens.shape)  # (cells,)

    # Center neutrino coords. on each cell center (whole grid).
    x_i = np.repeat(np.expand_dims(x_i, axis=(0,1)), len(cell_coords), axis=0)
    x_i -= cell_coords

    # All cell lengths. Limit for the largest cell is GRID_S/2, not just 
    # GRID_S, therefore the cell_gen+1 !
    cell_lens = np.expand_dims(init_GRID_S/(2**(cell_gens+1)), axis=1)
    # print(cell_lens.shape)  # (cells, 1)
    # print(x_i[...,0].shape)  # (cells, 1)

    # Find index of cell in which neutrino is enclosed.
    in_cell = np.asarray(
        (np.abs(x_i[...,0]) < cell_lens) & 
        (np.abs(x_i[...,1]) < cell_lens) & 
        (np.abs(x_i[...,2]) < cell_lens)
    )
    cell_idx = np.argwhere(in_cell==True).flatten()[0]  # np.int64

    return cell_idx


# Load files.
out_dir = 'L025N752/DMONLY/SigmaConstant00'
temp_dir = f'{out_dir}/temp_data_TEST'
cell_coords = np.load(f'{temp_dir}/fin_grid_origID28_snap_0014.npy')
cell_gens = np.load(f'{temp_dir}/cell_gen_origID28_snap_0014.npy')
init_GRID_S = np.load(f'{out_dir}/snaps_GRID_L.npy')[2]
print('Cells for this snap:', len(cell_coords))

with open(f'{out_dir}/sim_parameters.yaml', 'r') as file:
    sim_setup = yaml.safe_load(file)

init_dis = sim_setup['initial_haloGC_distance']
xE = np.cos(np.deg2rad(Pi))*np.sin(np.deg2rad(Pi))*init_dis
yE = np.sin(np.deg2rad(Pi))*np.sin(np.deg2rad(Pi))*init_dis
zE = np.cos(np.deg2rad(Pi))*init_dis
x_i = np.array([xE, yE, zE])*kpc

cell_idx = nu_in_which_cell(x_i, cell_coords, cell_gens, init_GRID_S)

In [ ]:
from itertools import zip_longest

x1 = np.arange(3)+6
x2 = np.arange(5)+6

y1 = np.arange(3)+25
y2 = np.arange(5)+25

lx1d = [x1,x2]
ly1d = [y1,y2]

arrx2d = np.array(list(zip_longest(*lx1d, fillvalue=np.nan))).T
arry2d = np.array(list(zip_longest(*ly1d, fillvalue=np.nan))).T

print(arrx2d)
print(arrx2d.shape)
print(arry2d)
print(arry2d.shape)

fin = np.stack((arrx2d, arry2d), axis=2)
print(fin)

In [ ]:
from itertools import product, combinations

arra = [[1,2],[3,4],[5,6]]
arrb = [4,5,6]
arrc = np.array(list(combinations(arra, 2)))
print(arrc)
print(np.product(arrc, axis=1))

In [ ]:
count_arr = np.arange(4)+2
print(count_arr)
count_arr[0] = 0.
count_arr[1] = 0.
count_arr[2] = 0.
# count_arr[3] = 0.
print(count_arr)

if np.all(count_arr==0):
    DM_axis0 = np.full(shape=(len(count_arr),1,3), fill_value=np.nan)
else:
    x_arr = np.arange(np.sum(count_arr))
    breaks = np.cumsum(count_arr[:-1])
    x_split = np.split(x_arr, breaks)
    # print(x_split)
    DM_axis0 = np.array(list(zip_longest(*x_split, fillvalue=np.nan))).T


print(DM_axis0)

In [ ]:
DM_count_mpoles = np.array([2])

x_arr = np.arange(np.sum(DM_count_mpoles))
breaks = np.cumsum(DM_count_mpoles[:-1])
x_split = np.split(x_arr, breaks)
print(x_split)
DM_axis0 = np.array(list(zip_longest(*x_split, fillvalue=np.nan))).T
DM_axis1 = np.array(list(zip_longest(*x_split, fillvalue=np.nan))).T
DM_axis2 = np.array(list(zip_longest(*x_split, fillvalue=np.nan))).T
print(DM_axis0)
DM_mpoles = np.stack((DM_axis0, DM_axis1, DM_axis2), axis=2)
print(DM_mpoles)

shapes/nums: 267 (267,) 1 (3,) (267, 3) (267,) 394.0 (1, 457240, 3) (267,) (457239, 2)

In [ ]:
from shared.preface import *
import shared.functions as fct

PRE = PRE(
    sim='L025N752', 
    z0_snap=36, z4_snap=13, DM_lim=1000,
    sim_dir=SIM_ROOT, sim_ver=SIM_TYPE,
    phis=20, thetas=20, vels=200,
    pre_CPUs=10, sim_CPUs=128
)


# Make temporary folder to store files, s.t. parallel runs don't clash.
rand_code = ''.join(
    random.choices(string.ascii_uppercase + string.digits, k=4)
)
TEMP_DIR = f'{PRE.OUT_DIR}/temp_data_{rand_code}'
os.makedirs(TEMP_DIR)

Testing = False
if Testing:
    mass_gauge = 12.3
    mass_range = 0.3
    size = 1
else:
    mass_gauge = 12.0
    mass_range = 0.6
    size = 10

hname = f'1e+{mass_gauge}_pm{mass_range}Msun'
fct.halo_batch_indices(
    PRE.Z0_STR, mass_gauge, mass_range, 'halos', size, 
    hname, PRE.SIM_DIR, TEMP_DIR
)
halo_batch_IDs = np.load(f'{TEMP_DIR}/halo_batch_{hname}_indices.npy')
halo_batch_params = np.load(f'{TEMP_DIR}/halo_batch_{hname}_params.npy')
halo_num = len(halo_batch_params)

print('********Number density band********')
print('Halo batch params (Rvir,Mvir,cNFW):')
print(halo_batch_params)
print('***********************************')

# Remove temporary folder with all individual neutrino files.
shutil.rmtree(TEMP_DIR)

In [ ]:
from shared.preface import *
import shared.functions as fct

# Initialize parameters and files.
PRE = PRE(
    sim='L012N376', 
    z0_snap=36, z4_snap=13, DM_lim=1000,
    sim_dir=SIM_ROOT, sim_ver=SIM_TYPE,
    phis=10, thetas=10, vels=100,
    pre_CPUs=6, sim_CPUs=6
)

Testing=False
if Testing:
    mass_gauge = 12.3
    mass_range = 0.3
    size = 1
else:
    mass_gauge = 12.0
    mass_range = 0.6
    size = 10

hname = f'1e+{mass_gauge}_pm{mass_range}Msun'
fct.halo_batch_indices(
    PRE.Z0_STR, mass_gauge, mass_range, 'halos', size, 
    hname, PRE.SIM_DIR, PRE.OUT_DIR
)
halo_batch_IDs = np.load(f'{PRE.OUT_DIR}/halo_batch_{hname}_indices.npy')
halo_batch_params = np.load(f'{PRE.OUT_DIR}/halo_batch_{hname}_params.npy')
halo_num = len(halo_batch_params)

print('********Number density band********')
print('Halo batch params (Rvir,Mvir,cNFW):')
print(halo_batch_params)
print('***********************************')

# Tests for CubeSpace components.

# Tests for functions in shared/functions.py

### Optimal momentum spacing.

In [ ]:
from shared.preface import *
import shared.functions as fct

Vs = 200
LOWER = 0.01*T_CNB
UPPER = 10*T_CNB

fig, ax = plt.subplots(1,1)

v_tot_min = 1/np.sqrt(NU_MASSES[-1]**2/LOWER**2 + 1) / (kpc/s)
v_tot_max = 1/np.sqrt(NU_MASSES[0]**2/UPPER**2 + 1) / (kpc/s)
v_lines = np.geomspace(v_tot_min, v_tot_max, 5)

v_tot_geo = np.geomspace(v_tot_min, v_tot_max, Vs)
ax.scatter(v_tot_geo, v_tot_geo+(1.1*v_tot_geo), s=2)

for v0, v1 in zip(v_lines[:-1], v_lines[1:]):
    v_segment = np.geomspace(v0, v1, int(Vs/4))
    ax.scatter(v_segment, v_segment, s=2)
    ax.axvline(v0, ls=':', c='r')

plt.xscale('log')
plt.yscale('log')
plt.show()

In [ ]:
from shared.preface import *
import shared.functions as fct


fig, ax = plt.subplots(1,1)

momenta = np.geomspace(0.01*T_CNB, 10*T_CNB, 100)
v_kpc = 1/np.sqrt(NU_MASS**2/momenta**2 + 1) / (kpc/s)  # rel. formula
ax.scatter(v_kpc, v_kpc, s=2)

# plt.xscale('log')
# plt.yscale('log')
plt.show()

### Conversions between momentum to velocity.

In [ ]:
from shared.preface import *
import shared.functions as fct

Vs = 100
LOWER = 0.01*T_CNB
UPPER = 400.*T_CNB

# Momentum range.
MOMENTA = np.geomspace(LOWER, UPPER, Vs)

# Without Lorentz factor.
v_mins = np.zeros(len(NU_MASSES))
v_maxs = np.zeros(len(NU_MASSES))
for i, m_nu in enumerate(NU_MASSES):
    v_km = MOMENTA / m_nu / (m/s)
    v_min, v_max = v_km[0], v_km[-1]
    # print(f'(m/s) -> v_min = {v_min:.2f}, v_max = {v_max:.2f}, %c = {v_max/const.c.value*100:.2f} : for {m_nu} eV neutrino')

    v_mins[i] = v_min
    v_maxs[i] = v_max

# With Lorentz factor.
v_mins = np.zeros(len(NU_MASSES))
v_maxs = np.zeros(len(NU_MASSES))
for m_nu in NU_MASSES:
    v_km = 1/np.sqrt(m_nu**2/MOMENTA**2 + 1) / (m/s)
    v_min, v_max = v_km[0], v_km[-1]
    print(f'(m/s) -> v_min = {v_min:.2f}, v_max = {v_max:.2f}, %c = {v_max/const.c.value*100:.2f} : for {m_nu} eV neutrino')

#! Since the sim is using 0.3 eV mass, the max. velocity present in the sim is 
#! ~20% of c, not the ~98% of the 0.01 eV neutrino (see output of cell).

# Back to momentum to check formulas & functions.

### Coordinates of Andromeda (AG) and the Virgo Cluster (VC).

In [ ]:
# Cross-check with values in Mertsch et al. (2020).
# -> x and y coords. are switched, since our setup is different,
# and my x (their y) coords. differ by 8.5 kpc due to placement of sun
# on our x-axis.
coords_VC = fct.halo_pos(GLAT_VC, GLON_VC, DIST_VC/kpc)
print(coords_VC, 'in kpc')
coords_AG = fct.halo_pos(GLAT_AG, GLON_AG, DIST_AG/kpc)
print(coords_AG, 'in kpc')

### Values in Table 1 of Mertsch et al. (2020).

In [ ]:
Rvir_Tab1 = fct.R_vir_fct(0, Mvir_MW)
print(Rvir_Tab1/kpc)
c_vir_Tab1 = fct.c_vir(0, Mvir_MW, Rvir_MW, Rs_MW)
Rs_Tab1 = Rvir_Tab1 / c_vir_Tab1
print(Rs_Tab1/kpc)

### Critical density of universe.

In [ ]:
rho_crit_today = fct.rho_crit(0)
print(f'{rho_crit_today*(Msun/kpc**3)/(kg/m**3):.2e} kg/m^3') 

### Time Variable s(z) and comparison to age of universe.

In [ ]:
# In s_of_z function we use
H0_mod = H0/ (1/s)
print(H0_mod)

test_z = 1
s_val = fct.s_of_z(test_z)
print(f'Value of time variable s in seconds at redhshift {test_z}:','\n', s_val)
print(
    'Age of universe comparison: \n', 
    f'"Observed/measured": {t0/s:.2e}, i.e. {t0/Gyr:.2f} Gyr \n', 
    f'What we use in s_of_z function: {1/H0/s:.2e}, i.e. {1/H0/Gyr:.2f} Gyr'
)

### Integrals for cosmic time.

In [ ]:
def t_integrand_a(a):

    # We need value of H0 in units of 1/s.
    H0_val = H0/(1/s)

    a_dot = np.sqrt(Omega_M/a**3 + Omega_L)*H0_val*a
    t_int = 1./a_dot

    return t_int

t, err = quad(t_integrand_a, 0, 1)
t_uni, err_uni = t, err
print(t_uni*s/Gyr, err_uni*s/Gyr)

In [ ]:
def t_integrand_z(z):

    # We need value of H0 in units of 1/s.
    H0_val = H0/(1/s)

    a_dot = np.sqrt(Omega_M*(1.+z)**3 + Omega_L)*H0_val*(1.+z)
    t_int = 1./a_dot

    return t_int

t, err = quad(t_integrand_z, 0, np.inf)
t_uni, err_uni = t, err
print(t_uni*s/Gyr, err_uni*s/Gyr)

### Fermi-Dirac distribution.

In [ ]:
p_test_range = np.linspace(0.01, 10)*T_CNB
FD_range = fct.Fermi_Dirac(p_test_range)

plt.loglog(p_test_range/T_CNB, FD_range)
plt.show()

### Redshift array for integration steps.

In [ ]:
# Linear spacing.
# late_steps = 200
# early_steps = 100
# Z_START, Z_STOP, Z_AMOUNT = 0., 4., late_steps+early_steps
# z_late = np.linspace(0,1,late_steps)
# z_early = np.linspace(1.01,4,early_steps)
# ZEDS = np.concatenate((z_late, z_early))

# Logarithmic spacing.
Z_AMOUNT = 50
z_shift = 1e-1
ZEDS = np.geomspace(z_shift, 4.+z_shift, Z_AMOUNT) - z_shift

plt.scatter(ZEDS, ZEDS, s=1)
plt.scatter(ZEDS_SNAPSHOTS, ZEDS_SNAPSHOTS+0.1, s=1)
plt.show()
print(ZEDS[0:10], ZEDS[-1])

### NFW density profile fct.

In [ ]:
r_range = np.geomspace(1e-3, 100, 100)*kpc
NFW_vals = fct.NFW_profile(r_range, rho0_MW, Rs_MW)
plt.loglog(r_range/kpc, NFW_vals/(GeV/cm**3))

### Typical grav. potential gradient vectors in spher. symmetric simulation.

In [ ]:
z = 0

# For position of Sun/Earth.
x_i = X_SUN*kpc
grad_MW = fct.dPsi_dxi_NFW(x_i, z, rho0_MW, Mvir_MW, Rvir_MW, Rs_MW, 'MW')
grad_MW /= (kpc/s**2)
print('Position of Sun/Earth:')
print(grad_MW)
print(np.sqrt(np.sum(grad_MW**2)), '\n')

# For a position closer to the center of the halo.
x_i = np.array([0.01, 0, 0])*kpc
grad_MW = fct.dPsi_dxi_NFW(x_i, z, rho0_MW, Mvir_MW, Rvir_MW, Rs_MW, 'MW')
grad_MW /= (kpc/s**2)
print('Position close to center:')
print(grad_MW)
print(np.sqrt(np.sum(grad_MW**2)), '\n')

# For a position further away to the center of the halo.
x_i = np.array([333., 0, 0])*kpc
grad_MW = fct.dPsi_dxi_NFW(x_i, z, rho0_MW, Mvir_MW, Rvir_MW, Rs_MW, 'MW')
grad_MW /= (kpc/s**2)
print('Position at virial radius:')
print(grad_MW)
print(np.sqrt(np.sum(grad_MW**2)))